In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

/var/folders/g2/dsm99_hd0c9_m_6qhbpwpvl80000gn/T/ipykernel_70120/3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
from IPython.display import display, HTML # Fix depracted instruction

# Configurar el ancho del contenedor
display(HTML("<style>.container { width:100% !important; }</style>"))

# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [7]:
dir_path = "../data/"

In [93]:
# File paths
train_file_path = f"{dir_path}kg_train.csv"  # Train DataSet
test_file_path = f"{dir_path}kg_test.csv"    # Test DataSet

# Load the training and test data
train_data = pd.read_csv(train_file_path, encoding='latin-1')
test_data = pd.read_csv(test_file_path, encoding='latin-1')

# Reduce the training set to speed up development (e.g., first 1000 rows)
train_data = train_data.head(1000)
test_data = test_data.head(1000)  # Fixed this line to reduce test_data, not train_data

# Fill missing values with an empty string to avoid errors during processing
train_data.fillna("", inplace=True)
test_data.fillna("", inplace=True)

# Display the shape and preview of the reduced dataset
print(f"Reduced Training Data Shape: {train_data.shape}")
print(train_data.head())

print(f"\nReduced Test Data Shape: {test_data.shape}")
print(test_data.head())


Reduced Training Data Shape: (1000, 2)
                                                text  label
0  DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...      1
1                                           Will do.      0
2  Nora--Cheryl has emailed dozens of memos about...      0
3  Dear Sir=2FMadam=2C I know that this proposal ...      1
4                                                fyi      0

Reduced Test Data Shape: (1000, 1)
                                                text
0  usiness is for the fact that the deceased man ...
1  They are happy to adjust to the afternoon. I a...
2  Lael Brainard was confirmed 78-19 this afterno...
3  H <hrod17@clintonemail.com>Friday March 26 201...
4  n;"> Dear Good Friend,<br><br><br>I am happy t...


### Let's divide the training and test set into two partitions

In [95]:
from sklearn.model_selection import train_test_split

# Split training data into two partitions: train_partition_1 and train_partition_2
train_partition_1, train_partition_2 = train_test_split(train_data, test_size=0.5, random_state=42)

# Split test data into two partitions: test_partition_1 and test_partition_2
test_partition_1, test_partition_2 = train_test_split(test_data, test_size=0.5, random_state=42)

# Display the shapes of the partitions
print(f"Train Partition 1 Shape: {train_partition_1.shape}")
print(f"Train Partition 2 Shape: {train_partition_2.shape}")
print(f"Test Partition 1 Shape: {test_partition_1.shape}")
print(f"Test Partition 2 Shape: {test_partition_2.shape}")

# Optional: Preview one of the partitions
print("\nPreview of Train Partition 1:")
print(train_partition_1.head())

Train Partition 1 Shape: (500, 2)
Train Partition 2 Shape: (500, 2)
Test Partition 1 Shape: (500, 1)
Test Partition 2 Shape: (500, 1)

Preview of Train Partition 1:
                                                  text  label
680  Please for more confidentiality of this transa...      1
177  Rich---Thanks for all you did to make the past...      0
395  K - will call in a bit - in line at dmv to sur...      0
911  Just wanted to tell you that speech yesterday ...      0
793  FROM MR JOHNSON MUGABE=2E E-MAIL =3B mm=5Fm=5F...      1


## Data Preprocessing

In [97]:
import string
from nltk.corpus import stopwords

print(string.punctuation)
print(stopwords.words("english")[100:110])

from nltk.stem.snowball import SnowballStemmer

snowball = SnowballStemmer('english')

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each']


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [14]:
import re

In [99]:
# Function to clean HTML content
def clean_html(text):
    # Remove inline JavaScript/CSS
    text = re.sub(r'<script.*?>.*?</script>', '', text, flags=re.DOTALL)  # Remove <script>...</script>
    text = re.sub(r'<style.*?>.*?</style>', '', text, flags=re.DOTALL)    # Remove <style>...</style>
    
    # Remove HTML comments
    text = re.sub(r'<!--.*?-->', '', text, flags=re.DOTALL)
    
    # Remove remaining HTML tags
    text = re.sub(r'<.*?>', '', text)
    
    return text

# Function to preprocess text
def preprocess_text(text):
    # Clean HTML
    text = clean_html(text)
    
    # Lowercase the text
    text = text.lower()
    
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # Remove stopwords
    stop_words = set(stopwords.words("english"))
    text = " ".join([word for word in text.split() if word not in stop_words])
    
    # Apply stemming
    text = " ".join([snowball.stem(word) for word in text.split()])
    
    return text

In [101]:
# Apply preprocessing to the 'text' column
train_partition_1['cleaned_text'] = train_partition_1['text'].apply(preprocess_text)

# Display the processed data
print(train_partition_1[['text', 'cleaned_text']].head())

                                                  text  \
680  Please for more confidentiality of this transa...   
177  Rich---Thanks for all you did to make the past...   
395  K - will call in a bit - in line at dmv to sur...   
911  Just wanted to tell you that speech yesterday ...   
793  FROM MR JOHNSON MUGABE=2E E-MAIL =3B mm=5Fm=5F...   

                                          cleaned_text  
680  pleas confidenti transact herebi advis kind re...  
177  richthank make past year success one hope fami...  
395          k call bit line dmv surrend ny licens sad  
911  want tell speech yesterday terrif import say n...  
793  mr johnson mugabe2 email 3b mm5fm5f17johnson40...  


- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [107]:
# Initialize the Snowball Stemmer
snowball = SnowballStemmer('english')

# Function to clean text
def clean_text(text):
    # Remove inline JavaScript/CSS
    text = re.sub(r'<script.*?>.*?</script>', '', text, flags=re.DOTALL)  # Remove <script>...</script>
    text = re.sub(r'<style.*?>.*?</style>', '', text, flags=re.DOTALL)    # Remove <style>...</style>
    
    # Remove HTML comments
    text = re.sub(r'<!--.*?-->', '', text, flags=re.DOTALL)
    
    # Remove remaining HTML tags
    text = re.sub(r'<.*?>', '', text)
    
    # Remove special characters
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    
    # Remove single characters
    text = re.sub(r'\b[a-zA-Z]\b', '', text)
    
    # Substitute multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text).strip()
    
    # Remove prefixed 'b' (if present due to byte representation)
    text = re.sub(r'^b\s+', '', text)
    
    # Convert text to lowercase
    text = text.lower()
    
    return text

# Function to preprocess text
def preprocess_text(text):
    # Step 1: Clean the text
    text = clean_text(text)
    
    # Step 2: Remove stopwords
    stop_words = set(stopwords.words("english"))
    text = " ".join([word for word in text.split() if word not in stop_words])
    
    # Step 3: Apply stemming
    text = " ".join([snowball.stem(word) for word in text.split()])
    
    return text

# Apply preprocessing to the 'text' column
train_partition_1['cleaned_text'] = train_partition_1['text'].apply(preprocess_text)

# Display the processed data
print(train_partition_1[['text', 'cleaned_text']].head())

                                                  text  \
680  Please for more confidentiality of this transa...   
177  Rich---Thanks for all you did to make the past...   
395  K - will call in a bit - in line at dmv to sur...   
911  Just wanted to tell you that speech yesterday ...   
793  FROM MR JOHNSON MUGABE=2E E-MAIL =3B mm=5Fm=5F...   

                                          cleaned_text  
680  pleas confidenti transact herebi advis kind re...  
177  richthank make past year success one hope fami...  
395            call bit line dmv surrend ny licens sad  
911  want tell speech yesterday terrif import say n...  
793  mr johnson mugabe email mmfmfjohnsonyahooecome...  


## Now let's work on removing stopwords
Remove the stopwords.

In [115]:
import re
import string
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

# Initialize the Snowball Stemmer
snowball = SnowballStemmer('english')

# Load stopwords from NLTK
stop_words = set(stopwords.words("english"))

# Function to clean text
def clean_text(text):
    # Remove inline JavaScript/CSS
    text = re.sub(r'<script.*?>.*?</script>', '', text, flags=re.DOTALL)  # Remove <script>...</script>
    text = re.sub(r'<style.*?>.*?</style>', '', text, flags=re.DOTALL)    # Remove <style>...</style>
    
    # Remove HTML comments
    text = re.sub(r'<!--.*?-->', '', text, flags=re.DOTALL)
    
    # Remove remaining HTML tags
    text = re.sub(r'<.*?>', '', text)
    
    # Remove special characters
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    
    # Remove all single characters
    text = re.sub(r'\b[a-zA-Z]\b', '', text)
    
    # Substitute multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text).strip()
    
    # Convert text to lowercase
    text = text.lower()
    
    return text

# Function to remove stopwords
def remove_stopwords(text):
    return " ".join([word for word in text.split() if word not in stop_words])

# Function to preprocess text
def preprocess_text(text):
    # Clean the text
    text = clean_text(text)
    
    # Remove stopwords
    text = remove_stopwords(text)
    
    # Apply stemming
    text = " ".join([snowball.stem(word) for word in text.split()])
    
    return text

# Apply preprocessing to the 'text' column
train_partition_1['cleaned_text'] = train_partition_1['text'].apply(preprocess_text)

# Display the processed data
print(train_partition_1[['text', 'cleaned_text']].head())

                                                  text  \
680  Please for more confidentiality of this transa...   
177  Rich---Thanks for all you did to make the past...   
395  K - will call in a bit - in line at dmv to sur...   
911  Just wanted to tell you that speech yesterday ...   
793  FROM MR JOHNSON MUGABE=2E E-MAIL =3B mm=5Fm=5F...   

                                          cleaned_text  
680  pleas confidenti transact herebi advis kind re...  
177  richthank make past year success one hope fami...  
395            call bit line dmv surrend ny licens sad  
911  want tell speech yesterday terrif import say n...  
793  mr johnson mugabe email mmfmfjohnsonyahooecome...  


## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [113]:
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

# Initialize the WordNet Lemmatizer
lemmatizer = WordNetLemmatizer()

# Load stopwords from NLTK
stop_words = set(stopwords.words("english"))

# Function to clean text
def clean_text(text):
    # Remove inline JavaScript/CSS
    text = re.sub(r'<script.*?>.*?</script>', '', text, flags=re.DOTALL)  # Remove <script>...</script>
    text = re.sub(r'<style.*?>.*?</style>', '', text, flags=re.DOTALL)    # Remove <style>...</style>
    
    # Remove HTML comments
    text = re.sub(r'<!--.*?-->', '', text, flags=re.DOTALL)
    
    # Remove remaining HTML tags
    text = re.sub(r'<.*?>', '', text)
    
    # Remove special characters
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    
    # Remove all single characters
    text = re.sub(r'\b[a-zA-Z]\b', '', text)
    
    # Substitute multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text).strip()
    
    # Convert text to lowercase
    text = text.lower()
    
    return text

# Function to remove stopwords
def remove_stopwords(text):
    return " ".join([word for word in text.split() if word not in stop_words])

# Function to apply lemmatization
def apply_lemmatization(text):
    words = word_tokenize(text)  # Tokenize the text into words
    return " ".join([lemmatizer.lemmatize(word) for word in words])

# Function to preprocess text
def preprocess_text(text):
    # Step 1: Clean the text
    text = clean_text(text)
    
    # Step 2: Remove stopwords
    text = remove_stopwords(text)
    
    # Step 3: Apply lemmatization
    text = apply_lemmatization(text)
    
    return text

# Apply preprocessing to the 'text' column
train_partition_1['cleaned_text'] = train_partition_1['text'].apply(preprocess_text)

# Display the processed data
print(train_partition_1[['text', 'cleaned_text']].head())

                                                  text  \
680  Please for more confidentiality of this transa...   
177  Rich---Thanks for all you did to make the past...   
395  K - will call in a bit - in line at dmv to sur...   
911  Just wanted to tell you that speech yesterday ...   
793  FROM MR JOHNSON MUGABE=2E E-MAIL =3B mm=5Fm=5F...   

                                          cleaned_text  
680  please confidentiality transaction hereby advi...  
177  richthanks make past year successful one hope ...  
395       call bit line dmv surrender ny license sadly  
911  wanted tell speech yesterday terrific importan...  
793  mr johnson mugabee email mmfmfjohnsonyahooecom...  


## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [127]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

# Separate spam and ham messages
ham_messages = train_partition_1[train_partition_1['label'] == 0]['text']
spam_messages = train_partition_1[train_partition_1['label'] == 1]['text']

# Define a function to preprocess and clean the text
def preprocess_text_simple(text):
    stop_words = set(stopwords.words("english"))
    # Lowercase, remove punctuation, and remove stopwords
    text = re.sub(r'[^a-zA-Z\s]', '', text).lower()
    text = " ".join([word for word in text.split() if word not in stop_words])
    return text

# Preprocess messages
ham_messages_cleaned = ham_messages.apply(preprocess_text_simple)
spam_messages_cleaned = spam_messages.apply(preprocess_text_simple)

# Bag of Words for Ham Messages
vectorizer_ham = CountVectorizer()
ham_bow = vectorizer_ham.fit_transform(ham_messages_cleaned)
ham_word_counts = pd.DataFrame({'word': vectorizer_ham.get_feature_names_out(),
                                'count': ham_bow.sum(axis=0).A1})
ham_top_words = ham_word_counts.sort_values(by='count', ascending=False).head(10)

# Bag of Words for Spam Messages
vectorizer_spam = CountVectorizer()
spam_bow = vectorizer_spam.fit_transform(spam_messages_cleaned)
spam_word_counts = pd.DataFrame({'word': vectorizer_spam.get_feature_names_out(),
                                 'count': spam_bow.sum(axis=0).A1})
spam_top_words = spam_word_counts.sort_values(by='count', ascending=False).head(10)

# Display Top Words
print("Top 10 Words in Ham Messages:")
print(ham_top_words)

print("\nTop 10 Words in Spam Messages:")
print(spam_top_words)

Top 10 Words in Ham Messages:
           word  count
4718         us     76
4979      would     69
3273         pm     63
4158      state     53
3393  president     52
3035        one     44
3200    percent     40
2792         mr     38
4961       work     36
2991     office     36

Top 10 Words in Spam Messages:
             word  count
5609        money    486
89        account    351
971          bank    318
9133           us    267
1405     business    237
3719         fund    208
5520      million    202
8844  transaction    199
8898     transfer    197
1838      company    186


## Extra features

In [129]:
print(train_partition_1.head())

                                                  text  label  \
680  Please for more confidentiality of this transa...      1   
177  Rich---Thanks for all you did to make the past...      0   
395  K - will call in a bit - in line at dmv to sur...      0   
911  Just wanted to tell you that speech yesterday ...      0   
793  FROM MR JOHNSON MUGABE=2E E-MAIL =3B mm=5Fm=5F...      1   

                                          cleaned_text  money_mark  \
680  pleas confidenti transact herebi advis kind re...           1   
177  richthank make past year success one hope fami...           1   
395            call bit line dmv surrend ny licens sad           1   
911  want tell speech yesterday terrif import say n...           1   
793  mr johnson mugabe email mmfmfjohnsonyahooecome...           1   

     suspicious_words  text_len  
680                 1      3989  
177                 0       707  
395                 0        81  
911                 0       580  
793               

In [131]:
# We add to the original dataframe two additional indicators (money symbols and suspicious words).
money_simbol_list = "|".join(["euro","dollar","pound","€","$"])
suspicious_words = "|".join(["free","cheap","sex","money","account","bank","fund","transfer","transaction","win","deposit","password"])

train_partition_1['money_mark'] = train_partition_1['text'].str.contains(money_simbol_list)*1
train_partition_1['suspicious_words'] = train_partition_1['text'].str.contains(suspicious_words)*1
train_partition_1['text_len'] = train_partition_1['text'].apply(lambda x: len(x)) 

train_partition_2['money_mark'] = train_partition_2['text'].str.contains(money_simbol_list)*1
train_partition_2['suspicious_words'] = train_partition_2['text'].str.contains(suspicious_words)*1
train_partition_2['text_len'] = train_partition_2['text'].apply(lambda x: len(x)) 

data_train.head()

,text,label,cleaned_text,money_mark,suspicious_words,text_len
0,"DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...",1,dear sir strictly private business proposal mi...,1,0,2292
1,Will do.,0,,1,0,8
2,Nora--Cheryl has emailed dozens of memos about...,0,noracheryl emailed dozen memo haiti weekend pl...,1,0,197
3,Dear Sir=2FMadam=2C I know that this proposal ...,1,dear sirfmadamc know proposal might surprise e...,1,1,2199
4,fyi,0,fyi,1,0,3


## How would work the Bag of Words with Count Vectorizer concept?

In [139]:
from sklearn.feature_extraction.text import CountVectorizer

# Initialize CountVectorizer
vectorizer = CountVectorizer()

# Fit the vectorizer to the data and transform the documents
X = vectorizer.fit_transform(train_partition_1['cleaned_text'])

# Get the feature names (vocabulary)
vocabulary = vectorizer.get_feature_names_out()

# Convert the matrix to an array for better readability
word_counts = X.toarray()

# Display results
print("Vocabulary:", vocabulary)
print("\nBag of Words Representation (Count Matrix):")
print(word_counts)

Vocabulary: ['aaronovitchon' 'abacha' 'abachabefor' ... 'zuma' 'zumadirector' 'zurich']

Bag of Words Representation (Count Matrix):
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


## TD-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [149]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize the TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform the entire dataset
tfidf_matrix = tfidf_vectorizer.fit_transform(train_partition_1['cleaned_text'])

# Print the shape of the TF-IDF vectorized dataset
print(f"Shape of the vectorized dataset: {tfidf_matrix.shape}")

# Optionally, you can inspect the feature names (vocabulary)
vocabulary = tfidf_vectorizer.get_feature_names_out()
print(f"Vocabulary size: {len(vocabulary)}")

# Applying Stopwords
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
# Fit and transform the entire dataset
tfidf_matrix = tfidf_vectorizer.fit_transform(train_partition_1['cleaned_text'])

# Print the shape of the TF-IDF vectorized dataset
print(f"Shape of the vectorized dataset Removing Stopwords Technique: {tfidf_matrix.shape}")

# Applying N-grams
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2))  # Unigrams and bigrams
# Fit and transform the entire dataset
tfidf_matrix = tfidf_vectorizer.fit_transform(train_partition_1['cleaned_text'])

# Print the shape of the TF-IDF vectorized dataset
print(f"Shape of the vectorized dataset with N-Grams Technique: {tfidf_matrix.shape}")

# Applying Limit Vocabulary Size
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2))  # Unigrams and bigrams
# Fit and transform the entire dataset
tfidf_matrix = tfidf_vectorizer.fit_transform(train_partition_1['cleaned_text'])

# Print the shape of the TF-IDF vectorized dataset
print(f"Shape of the vectorized dataset with Limit Vocabulary Size Technique: {tfidf_matrix.shape}")

Shape of the vectorized dataset: (500, 10761)
Vocabulary size: 10761
Shape of the vectorized dataset Removing Stopwords Technique: (500, 10621)
Shape of the vectorized dataset with N-Grams Technique: (500, 46536)
Shape of the vectorized dataset with Limit Vocabulary Size Technique: (500, 46536)


## And the Train a Classifier?

In [153]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

# Step 1: Initialize TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Step 2: Vectorize the Dataset
X_train_tfidf = tfidf_vectorizer.fit_transform(train_partition_1['text'])
X_test_tfidf = tfidf_vectorizer.transform(train_partition_2['text'])

# Step 3: Extract Target (Labels)
y_train = train_partition_1['label']
y_test = train_partition_2['label']

# Step 4: Train a Classifier (e.g., Logistic Regression)
classifier = LogisticRegression(random_state=42)
classifier.fit(X_train_tfidf, y_train)

# Step 5: Make Predictions
y_pred = classifier.predict(X_test_tfidf)

# Step 6: Evaluate the Classifier
print("Classification Report:")
print(classification_report(y_test, y_pred))

print(f"Accuracy: {accuracy_score(y_test, y_pred):.2f}")

Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.98      0.94       283
           1       0.97      0.87      0.92       217

    accuracy                           0.93       500
   macro avg       0.94      0.92      0.93       500
weighted avg       0.94      0.93      0.93       500

Accuracy: 0.93


### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to find the **best feature representation**.

You can work with teams of two persons (recommended).

In [ ]:
#!kaggle competitions download -c dsub-fraudulentemails

In [216]:
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score

# Preprocessing function
def preprocess_text(text):
    if not isinstance(text, str):
        text = str(text)
    text = text.lower()
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Load the dataset
dataset_path = f"{dir_path}kg_train.csv"  # Replace with your dataset path
data = pd.read_csv(dataset_path, encoding='latin-1')

# Debug: Inspect dataset structure
print("Dataset columns:", data.columns)
print("First few rows of the dataset:")
print(data.head())

# Skip mapping as the labels are already numeric
# Debug: Check unique values in the 'label' column
print("Unique values in 'label':", data['label'].unique())

# Fill missing values in the 'text' column with an empty string
data['text'] = data['text'].fillna('')

# Apply preprocessing to the text
data['cleaned_text'] = data['text'].apply(preprocess_text)

# Remove rows with empty cleaned_text
data = data[data['cleaned_text'].str.strip() != '']

# Ensure the dataset is not empty
if data.shape[0] == 0:
    raise ValueError("The dataset is empty after preprocessing.")

# Print dataset shape after preprocessing
print(f"Dataset shape: {data.shape}")

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    data['cleaned_text'], data['label'], test_size=0.2, random_state=42
)

# Check for null values in y_train and y_test
print(f"Null values in y_train: {y_train.isnull().sum()}")
print(f"Null values in y_test: {y_test.isnull().sum()}")

# Feature Representation
# Bag of Words
count_vectorizer = CountVectorizer(ngram_range=(1, 2), max_features=5000)
X_train_count = count_vectorizer.fit_transform(X_train)
X_test_count = count_vectorizer.transform(X_test)

# TF-IDF
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_features=5000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Train and Evaluate MultinomialNB
# Bag of Words
nb_count = MultinomialNB()
nb_count.fit(X_train_count, y_train)
y_pred_count = nb_count.predict(X_test_count)

print("Results with Bag of Words (CountVectorizer):")
print(classification_report(y_test, y_pred_count))
print(f"Accuracy: {accuracy_score(y_test, y_pred_count):.2f}")

# TF-IDF
nb_tfidf = MultinomialNB()
nb_tfidf.fit(X_train_tfidf, y_train)
y_pred_tfidf = nb_tfidf.predict(X_test_tfidf)

print("\nResults with TF-IDF (TfidfVectorizer):")
print(classification_report(y_test, y_pred_tfidf))
print(f"Accuracy: {accuracy_score(y_test, y_pred_tfidf):.2f}")

# Compare and Select Best Representation
bag_accuracy = accuracy_score(y_test, y_pred_count)
tfidf_accuracy = accuracy_score(y_test, y_pred_tfidf)

print(f"Accuracy Bag of Words: {bag_accuracy:.2f}")
print(f"Accuracy TF-IDF: {tfidf_accuracy:.2f}")

if bag_accuracy > tfidf_accuracy:
    print("\nBest Feature Representation: Bag of Words (CountVectorizer)")
else:
    print("\nBest Feature Representation: TF-IDF (TfidfVectorizer)")

Dataset columns: Index(['text', 'label'], dtype='object')
First few rows of the dataset:
                                                text  label
0  DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...      1
1                                           Will do.      0
2  Nora--Cheryl has emailed dozens of memos about...      0
3  Dear Sir=2FMadam=2C I know that this proposal ...      1
4                                                fyi      0
Unique values in 'label' column before mapping:
[1 0]
Unique values in 'label' column after mapping:
[nan]
Dataset shape after dropping NaN labels: (0, 2)


ValueError: The dataset is empty after preprocessing.